In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import os

from sklearn.cluster import MeanShift
from matplotlib.pyplot import figure
from scipy.spatial import ConvexHull
from matplotlib.cm import get_cmap
from scipy import interpolate
from matplotlib import cm

In [ ]:
import math
import numpy as np
import pandas as pd


class EdgePathBundling:
    newton = dict()

    def __init__(self, nodes: pd.DataFrame, edges: pd.DataFrame, color_list=None, color_map=None,
                 number_control_points=3, num_approximation_points=50, arrow_size=25, width=0, height=0):
        if color_list is None:
            color_list = []
        self.num_approximation_points = num_approximation_points
        self.number_control_points = number_control_points
        self.arrow_size = arrow_size
        self.color_list = color_list
        self.color_map = color_map
        self.multiply_dist = 0.25
        if len(self.color_list) == 0 and self.color_map is None:
            raise ValueError("One of color mapping must contains data")

        self.nodes = nodes
        self.edges = edges

        self.node_dict = {}
        self.min_x = self.nodes['x'].min()
        self.min_y = self.nodes['y'].min()
        self.max_x = self.nodes['x'].max()
        self.max_y = self.nodes['y'].max()

        for i, v in self.nodes.iterrows():
            self.node_dict[i] = v['x'], v['y']

        self._scale(width, height)

        bezier_points_x = np.empty((0, self.num_approximation_points + 1))
        bezier_points_y = np.empty((0, self.num_approximation_points + 1))
        colors = []
        arrows = []
        for _, edge in self.edges.iterrows():
            st = edge['start']
            nd = edge['end']
            str_node = self.nodes.loc[st]
            end_node = self.nodes.loc[nd]
            cor_x, cor_y = self._approximate_bezier(str_node, end_node)
            bezier_points_x = np.append(bezier_points_x, np.array([cor_x]), axis=0)
            bezier_points_y = np.append(bezier_points_y, np.array([cor_y]), axis=0)
            colors.append(self._assign_color(str_node, end_node, edge['weight']))
            arrows.append(self.arrow(cor_x, cor_y))
        self.edges['arrows'] = arrows
        self.edges['color'] = colors
        self.edges['x'] = [bezier_points_x[i] for i in range(len(bezier_points_x))]
        self.edges['y'] = [bezier_points_y[i] for i in range(len(bezier_points_y))]

    def _scale(self, maxX, maxY):
        if maxX != 0 and maxY != 0:
            w = self.max_x - self.min_x
            h = self.max_y - self.min_y
            marginX = maxX * 0.05
            marginY = maxY * 0.05
            maxX -= 2.0 * marginX
            maxY -= 2.0 * marginY
            self.nodes.x = marginX + ((self.nodes.x - self.min_x) / w) * maxX
            self.nodes.y = marginY + ((self.nodes.y - self.min_y) / h) * maxY

    def _approximate_bezier(self, src, tgt):
        dt = 1.0 / self.num_approximation_points
        base_control_points = self._approximate_straight(src, tgt, self.number_control_points)
        control_points = self._generate_controls(src, tgt, base_control_points, self.multiply_dist)
        coordinates = [(self._point(i * dt, control_points)) for i in range(self.num_approximation_points + 1)]
        x = np.array([coordinates[i][0] for i in range(len(coordinates))])
        y = np.array([coordinates[i][1] for i in range(len(coordinates))])
        return x, y

    def _generate_controls(self, src, tgt, base, dist):
        y_diff = 0.00001 if tgt.y == src.y else tgt.y - src.y
        a = (tgt.x - src.x) / y_diff
        a_inv = 9999999 if a == 0.0 else -1.0 / a
        dx = (tgt.x - src.x) / (self.number_control_points + 1)
        dy = (tgt.y - src.y) / (self.number_control_points + 1)
        d2 = (dx ** 2) + (dy ** 2)
        sqr = ((dist * d2) / (a_inv ** 2 + 1)) ** 0.5
        point_list = [(src.x, src.y)]
        switch = 1.0
        for i in range(len(base)):
            b_inv = base[i][1] - a_inv * base[i][0]
            x_control = switch * sqr + base[i][0]
            y_control = a_inv * x_control + b_inv
            point_list.append((x_control, y_control))
            switch = -1.0 * switch
        point_list.append((tgt.x, tgt.y))
        return point_list

    def _point(self, t, controls):
        x = 0.0
        y = 0.0
        n = len(controls) - 1
        for i in range(n + 1):
            b = self._b(n, i, t)
            x += controls[i][0] * b
            y += controls[i][1] * b
        return x, y

    def _approximate_straight(self, src, tgt, n):
        dx = (tgt.x - src.x) / (n + 1)
        dy = (tgt.y - src.y) / (n + 1)
        return [(src.x + i * dx, src.y + i * dy) for i in range(1, n + 1)]

    def arrow(self, cor_x, cor_y):
        length = len(cor_x)
        tgt = (cor_x[length-1], cor_y[length-1])
        src = (cor_x[length-2], cor_y[length-2])
        degree = (self._angle(src, tgt) + 180.0) % 360.0
        degree_l = math.radians(degree + 30.0)
        point_l = (
            self.arrow_size * math.cos(degree_l) + tgt[0],
            self.arrow_size * math.sin(degree_l) + tgt[1]
        )
        degree_r = math.radians(degree - 30.0)
        point_r = (
            self.arrow_size * math.cos(degree_r) + tgt[0],
            self.arrow_size * math.sin(degree_r) + tgt[1]
        )
        left = [[point_l[0], tgt[0]], [point_l[1], tgt[1]]]
        rigt = [[point_r[0], tgt[0]], [point_r[1], tgt[1]]]
        return [left, rigt]

    def _angle(self, src, tgt):
        dX = tgt[0] - src[0]
        dY = tgt[1] - src[1]

        angle = math.atan2(dY, dX)
        degrees = 180 * angle / math.pi

        if degrees < 0:
            degrees = 360 + round(degrees, 2)
        return degrees

    def _assign_color(self, src, tgt, width=1):
        dX = tgt.x - src.x
        dY = tgt.y - src.y
        degrees = 180 * math.atan2(dY, dX) / math.pi
        if degrees < 0:
            degrees = 360 + round(degrees, 2)

        if self.color_map is not None:
            return self.color_map(degrees / 360.0)

        len_colors = len(self.color_list)
        if len_colors == 0:
            return "#000000"
        arc = 360.0 / len_colors
        return dict(enumerate(self.color_list))[degrees // arc]

    def _newton(self, n, k):
        if self.newton.__contains__((n, k)):
            return self.newton[(n, k)]
        res = 1
        for i in range(1, k + 1):
            res = res * (n - i + 1) / i
        self.newton[(n, k)] = res
        return res

    def _b(self, n, i, t):
        return self._newton(n, i) * (t ** i) * (1.0 - t) ** (n - i)

In [ ]:
full_path = r'C:\Users\PC\Desktop\SEMESTER_5\OBO\analizy_wlasciwe\P09\csv\et_fixations_acc.csv'
img_path = os.chdir( r'C:\Users\PC\Desktop\SEMESTER_5\OBO\analizy_wlasciwe\P09\photos\edit')
df = pd.read_csv(full_path)

temp_path = r'C:/Users/PC/Desktop/SEMESTER_5/OBO/analizy_wlasciwe/wyniki/'

list_group = df['group'].unique()

for filename in os.listdir(img_path):
    img_path = os.chdir( r'C:\Users\PC\Desktop\SEMESTER_5\OBO\analizy_wlasciwe\P09\photos\edit')
    img = plt.imread(filename)
    txt = filename
    x = txt[:-4].split("_")
    obiekt = x[0]
    zdjecie = x[1]
    
    for group in list_group:
        
        df = pd.read_csv(full_path)
        df_output_without_time = pd.DataFrame(columns = [['AOI_label', 'fix', 'fdur', 'dwell']])

        df_cerkiew_s01 = df[(df['obiekt'] == obiekt) & (df['zdjecie'] == zdjecie) & (df['group'] == group)]
        df_cerkiew_s01.set_index('Order')
        df_cerkiew_s01 = df_cerkiew_s01.drop(df_cerkiew_s01[df_cerkiew_s01.mean_x < 200].index)
        df_cerkiew_s01 = df_cerkiew_s01.drop(df_cerkiew_s01[df_cerkiew_s01.mean_y < 0].index)
        #tworzymy dodatkowe dwie puste kolumny dla współrzędnych końca sakady (czyli punktu fiksacyjnego kolejnego w kolejności patrzenia)
        df_cerkiew_s01['mean_x_k'] = 0
        df_cerkiew_s01['mean_y_k'] = 0

        for n in range(0, len(df_cerkiew_s01)-1):
            df_cerkiew_s01['mean_x_k'].iloc[n] = df_cerkiew_s01['mean_x'].iloc[n+1]
            df_cerkiew_s01['mean_y_k'].iloc[n] = df_cerkiew_s01['mean_y'].iloc[n+1]
        df_cerkiew_s01 = df_cerkiew_s01[:-1]


        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)
        sns.scatterplot(df_cerkiew_s01.mean_x_k, df_cerkiew_s01.mean_y_k, alpha = 0.7)
        ax.set_ylim(ymin=0, ymax = 1050)
        ax.set_xlim(xmin=200)
        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()
        
        #musiałbym tworzyć ścieżkę na podstawie tego co jest + nazwa zdjęcia + grupa jako string i przekazywać to do os.chdir by ustawić tam ścieżkę i tam to zapisywać
        folder = obiekt + '_' + zdjecie + "_" + group  + "/"
        cur_path = temp_path + folder
        os.makedirs(os.path.dirname(cur_path), exist_ok=True)
        os.chdir(cur_path)
        
        name = obiekt + '_scatter_' + group + '.jpg'
        fig.savefig(name)

        #tworzymy nowy DataFrame w celu przechowywania tylko istotnych danych dla analiz, na podstawie poprzedniego przefiltrowanego DataFrame'a
        column_names = []
        df_pozitions = pd.DataFrame(columns = column_names)

        df_pozitions['mean_x_k'] = round(df_cerkiew_s01['mean_x'], 2) 
        df_pozitions['mean_y_k'] = round(df_cerkiew_s01['mean_y'], 2) 
        #df_pozitions['Order'] = df_D_casa['Order']
        df_pozitions = df_pozitions.dropna()

        #df_pozitions.set_index('Order')

        clustering = MeanShift(bandwidth=150).fit(df_pozitions)
        df_pozitions['klaster'] = clustering.labels_ #dodajemy kolumne z ID klastra
        #następnie dodajemy do DF'a na którym pracujemy wartości potrzebne do optymalizacji przerw naturalnych - do klasteryzacji w wymiarze czasu
        df_pozitions['Duration'] = df_cerkiew_s01['Duration'] 
        df_pozitions['Start'] = df_cerkiew_s01['Start'] 
        df_pozitions['End'] = df_cerkiew_s01['End'] 
        
        #tworznie CSV
        fix = df_pozitions['klaster'].value_counts()
        dwell = df_pozitions.groupby(['klaster'])['Duration'].agg('sum')
        fdur = dwell / fix


        df_output_without_time['fix'] = fix.sort_index()
        df_output_without_time['dwell'] = dwell.sort_index()
        df_output_without_time['fdur'] = fdur.sort_index()
        df_output_without_time['AOI_label'] = df_output_without_time.index
        df_output_without_time.reset_index().drop(['index'], axis = 1)
        #tu jeszcze zapisać CSV
        df_output_without_time.to_csv('df_output_without_time.csv')

        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)

        #wizyalizacja po klateryzacji w wymiarze przestrzeni
        sns.scatterplot(df_pozitions.mean_x_k, df_pozitions.mean_y_k, hue = df_pozitions.klaster, palette = 'dark', legend = True)
        #jeśli nie zadziała, to przeładować komórkę wyżej jeszcze raz, bo z jakiegoś nieznanego mi powodu nie odświeża danych



        ax.set_ylim(ymin=0, ymax = 1050)
        ax.set_xlim(xmin=200)
        ax.get_legend().remove()
        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()
        
        name = obiekt + '_catter_cluster_' + group + '.jpg'
        fig.savefig(name)


        df_start_time = df_pozitions.groupby('klaster')['Start'].agg(['sum', 'mean', 'std']).round(1)

        slownik = dict()
        for index, row in df_pozitions.iterrows():
            k = row['klaster']
            if k not in slownik:
                slownik[k] = list()
            slownik[k].append((row['mean_x_k'], row['mean_y_k'])) 

        slownik

        center = pd.DataFrame(columns = ['klaster', 'x', 'y'])
        for i, v in slownik.items():
            x = 0
            y = 0
            for xy in v:
                x += xy[0]
                y += xy[1]
            xc = x/len(v)
            yc = y/len(v)
            center.loc[i] = np.array([i, xc, yc])

        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)
        plt.scatter(df_pozitions.mean_x_k, df_pozitions.mean_y_k, c=df_pozitions.klaster, alpha = 0.5, s=20)
        #sns.scatterplot(df_pozitions.mean_x_k, df_pozitions.mean_y_k, hue = df_pozitions.klaster, palette = 'dark', legend = False)
        plt.scatter(center.x, center.y, c='g', s=1000, alpha = 0.6)

        for d in range(len(df_start_time)):
            text = 'Mean: ' + str(df_start_time['mean'][d]) +'sec' '\n' 'Std: ' + str(df_start_time['std'][d])  +'sec' 
            plt.text(x = center.x[d]-75, y = center.y[d]+10, s = text, fontdict = dict(color = 'red', size = 8))
            plt.text(x = center.x[d]+30, y = center.y[d]-50, s = df_start_time.index[d], fontdict = dict(color = 'blue', size = 15))

        name = "tab20b"
        cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
        colors = cmap.colors  # type: list
        ax.set_prop_cycle(color=colors)

        for i in df_pozitions.klaster.unique():
            # get the convex hull
            if len(df_pozitions[df_pozitions.klaster == i][['mean_x_k', 'mean_y_k']].values) > 4:
                points = df_pozitions[df_pozitions.klaster == i][['mean_x_k', 'mean_y_k']].values
                hull = ConvexHull(points)
                x_hull = np.append(points[hull.vertices,0],
                                   points[hull.vertices,0][0])
                y_hull = np.append(points[hull.vertices,1],
                                   points[hull.vertices,1][0])

                # interpolate
                dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
                dist_along = np.concatenate(([0], dist.cumsum()))
                spline, u = interpolate.splprep([x_hull, y_hull], 
                                                u=dist_along, s=0)
                interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
                interp_x, interp_y = interpolate.splev(interp_d, spline)
                # plot shape
                plt.fill(interp_x, interp_y, '--', alpha=0.6)
                plt.plot(interp_x, interp_y, alpha=0.8, linewidth = 2)


        ax.set_ylim(ymin=0, ymax = 1050)
        ax.set_xlim(xmin=200)
        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()

        name = obiekt + '_scatter_polygon_' + group + '.jpg'
        fig.savefig(name)


        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)
        #plt.scatter(df_pozitions.mean_x_k, df_pozitions.mean_y_k, c=df_pozitions.klaster, alpha = 0.9, s=20)
        #sns.scatterplot(df_pozitions.mean_x_k, df_pozitions.mean_y_k, hue = df_pozitions.klaster, palette = 'dark', legend = False)
        plt.scatter(center.x, center.y, c='g', s=1000, alpha = 0.7)

        for d in range(len(df_start_time)):
            text = 'Mean: ' + str(df_start_time['mean'][d]) +'sec' '\n' 'Std: ' + str(df_start_time['std'][d])  +'sec' 
            plt.text(x = center.x[d]-75, y = center.y[d]+10, s = text, fontdict = dict(color = 'red', size = 8))
            plt.text(x = center.x[d]+30, y = center.y[d]-50, s = df_start_time.index[d], fontdict = dict(color = 'blue', size = 15))

        name = "tab20b"
        cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
        colors = cmap.colors  # type: list
        ax.set_prop_cycle(color=colors)

        for i in df_pozitions.klaster.unique():
            # get the convex hull
            if len(df_pozitions[df_pozitions.klaster == i][['mean_x_k', 'mean_y_k']].values) > 4:
                points = df_pozitions[df_pozitions.klaster == i][['mean_x_k', 'mean_y_k']].values
                hull = ConvexHull(points)
                x_hull = np.append(points[hull.vertices,0],
                                   points[hull.vertices,0][0])
                y_hull = np.append(points[hull.vertices,1],
                                   points[hull.vertices,1][0])

                # interpolate
                dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
                dist_along = np.concatenate(([0], dist.cumsum()))
                spline, u = interpolate.splprep([x_hull, y_hull], 
                                                u=dist_along, s=0)
                interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
                interp_x, interp_y = interpolate.splev(interp_d, spline)
                # plot shape
                plt.fill(interp_x, interp_y, '--', alpha=0.4)
                plt.plot(interp_x, interp_y, alpha=0.8, linewidth = 2)

        ax.set_ylim(ymin=0)
        ax.set_xlim(xmin=200)
        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()

        name = obiekt + '_polygon_' + group + '.jpg'
        fig.savefig(name)


        #To jest podstawa do analiz Levensteina
        df_start_time_sorted = df_start_time.sort_values(by = 'mean')
        #display(df_start_time_sorted)
        #do listy
        naive_leven_list = df_start_time_sorted.index.tolist()

        with open("naive_leven.txt", "w") as text_file:
            for item in naive_leven_list:
                text_file.write("%s\n" % item)


        def std(x): return np.std(x)
        df_pogrupwoae_srednia = df_pozitions.groupby(['klaster'])['Start'].agg(['mean', std]).sort_values(by = 'mean').round(0)

        list_to_merge = list()

        for i in range(0, len(df_pogrupwoae_srednia)):

            test = df_pozitions[df_pozitions['klaster'] == i]
            points = list(test['Start'])

            clusters = []
            #eps =  0.5 * df_pogrupwoae_srednia['std'][i]
            eps = df_pogrupwoae_srednia['std'][i] #wybrane empisrycznmie, wcześniej std, ale rozdzielało zmienne które czasowo są bardzo blisko siebie, do dyskusji
            points_sorted = sorted(points)
            curr_point = points_sorted[0]
            curr_cluster = [curr_point]
            for point in points_sorted[1:]:
                if point <= curr_point + eps:
                    curr_cluster.append(point)
                else:
                    clusters.append(curr_cluster)
                    curr_cluster = [point]
                curr_point = point
            clusters.append(curr_cluster)
            nazwy = list(range(1, len(clusters) + 1))
            dic = dict(zip(nazwy, clusters))

            for key, values in dic.items():    
                for value in values:
                    list_to_merge.append(key)
                    list_to_merge.append(value)

        #print(list_to_merge)

        #łączymy klastry w domeneie czasu z naszym DF'em na którym pracyjemy cały czas
        output_array = np.array(list_to_merge).reshape(-1,2)
        df_with_time_claster = df_pozitions.merge(pd.DataFrame(output_array, columns = ['time_klaster', 'Start']))

        #powstaje nam DF, który ma w sobie klastry w domenie pozycyjnej i klastry w domenie czasowej
        #df_with_time_claster

        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)
        #wizualizujemy klastry w domenie czasoso-pozycyjnej - kolor oznacza klaster pozycyjny, w którym rozróżniamy za pomocą kształtów również podklastry w domenie czasu
        sns.scatterplot(df_with_time_claster.mean_x_k, df_with_time_claster.mean_y_k, hue = df_with_time_claster.klaster, palette = 'dark', style = df_with_time_claster.time_klaster, legend = False)


        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()
        
        name = obiekt + '_cluter_by_time_' + group + '.jpg'
        fig.savefig(name)

       

        #prosta operacja na nazwach klastrów, żeby ujednolicić nazewnictwo i mieć w jednej nazwie zawartą informację o tym do jakiego klastra pozycyjnego i czasowego należy rekord
        df_with_time_claster['finish_claster'] = df_with_time_claster.apply(lambda row: int(row.klaster) * 100 + int(row.time_klaster), axis =1)
        df_kolejnosc = df_with_time_claster.groupby(['finish_claster'])['Start'].min().sort_values() #ustalenie kolejności przebiegu patrzenia
        df_start_time_finish_claster = df_with_time_claster.groupby('finish_claster')['Start'].agg(['sum', 'mean', 'std']).round(1)
        
        #tworzenie CSV
        df_output_with_time = pd.DataFrame(columns = [['AOI_label', 'fix', 'fdur', 'dwell']])
        labels = df_with_time_claster['finish_claster'].unique()
        fix = df_with_time_claster['finish_claster'].value_counts().sort_index()
        dwell = df_with_time_claster.groupby(['finish_claster'])['Duration'].agg('sum').sort_index()
        fdur = (dwell / fix).sort_index()


        df_output_with_time['fix'] = fix
        df_output_with_time['dwell'] = dwell
        df_output_with_time['fdur'] = fdur
        df_output_with_time['AOI_label'] = df_output_with_time.index
        df_output_with_time.reset_index().drop(['index'], axis = 1)
        #zapisywanie CSV
        df_output_with_time.to_csv('df_output_with_time.csv')
        
        df_start_time_finish_claster_sorted = df_start_time_finish_claster.sort_values(by = 'mean')
        #display(df_start_time_finish_claster_sorted)
        #do listy
        leven_list = df_start_time_finish_claster_sorted.index.tolist()
        
        with open("leven.txt", "w") as text_file:
            for item in leven_list:
                text_file.write("%s\n" % item)

        df_kolejnosc.reset_index()
        slownik = dict()
        for index, row in df_with_time_claster.iterrows():
            k = row['finish_claster']
            if k not in slownik:
                slownik[k] = list()
            slownik[k].append((row['mean_x_k'], row['mean_y_k'])) 

        df = pd.DataFrame(columns = ['finish_claster', 'x', 'y'])
        for i, v in slownik.items():
            x = 0
            y = 0
            for xy in v:
                x += xy[0]
                y += xy[1]
            xc = x/len(v)
            yc = y/len(v)
            df.loc[i] = np.array([i, xc, yc])

        df['x'] = pd.to_numeric(df['x'])
        df['y'] = pd.to_numeric(df['y'])
        df_to_R = df_kolejnosc.to_frame().merge(df, on = 'finish_claster')       


        fig, ax = plt.subplots(1, figsize=(15, 10), dpi=100)

        sns.scatterplot(x = df.x, y = df.y, s = 1000, alpha = 0.5, color = 'g')
        sns.scatterplot(df_with_time_claster.mean_x_k, df_with_time_claster.mean_y_k, hue = df_with_time_claster.klaster, palette = 'dark', style = df_with_time_claster.time_klaster, alpha = 0.15, legend = False)
        for line in range(0, df_to_R.shape[0]):
            plt.text(x = df_to_R.x[line]-30, y = df_to_R.y[line]-10, s = df_to_R.index[line], fontdict = dict(color = 'red', size = 7))

        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()

        name = obiekt + '_centroid_' + group + '.jpg'
        fig.savefig(name)



        df_to_R['finish_claster_k'] = 0 #dodanie pustej kolumny dla klastra w którym zagregowana ściezka patrzenia kończy bieg
        for n in range(0, len(df_to_R)-1):
            df_to_R['finish_claster_k'].iloc[n] = df_to_R['finish_claster'].iloc[n+1]

        df_egdes = pd.DataFrame(df_to_R, columns = ['finish_claster', 'finish_claster_k'])[:-1]
        df_egdes.head()

        #tworzymy nodes
        nodes = pd.DataFrame(columns = ['id', 'x', 'y'])

        for _, node in df_to_R.iterrows():
            nodes.loc[len(nodes.index)] = [int(node['finish_claster']), float(node['x']), float(node['y'])]
        nodes.index = nodes['id']

        nodes.dtypes

        #tworzymy edges
        edges = pd.DataFrame(columns = ['start', 'end', 'weight'])

        for _, edge in df_egdes.iterrows():
            edges.loc[len(edges.index)] = [int(edge['finish_claster']), int(edge['finish_claster_k']), 1]

        edges.dtypes


        cmap = cm.get_cmap('hsv')
        nazwa_1 = EdgePathBundling(nodes, edges, number_control_points = 2, color_map = cmap)

        fig = plt.figure(figsize=(15, 10), dpi=100)

        for i in range(len(nazwa_1.edges)):
            plt.plot(nazwa_1.edges['x'][i], nazwa_1.edges['y'][i], color = nazwa_1.edges['color'][i], alpha = 0.8, linewidth = 4)
            plt.plot(nazwa_1.edges['x'][i], nazwa_1.edges['y'][i], color = 'w', alpha = 0.3, linewidth = 2)
            arrows = nazwa_1.edges['arrows'][i]
            for j in range(2):
                ax.plot(arrows[j][0], arrows[j][1], color = nazwa_1.edges['color'][i], linewidth = 4)
                ax.plot(arrows[j][0], arrows[j][1], color = 'w', alpha = 0.3, linewidth = 2)
       

        sns.scatterplot(x = df.x, y = df.y, s = 2000, alpha = 0.4, color = 'g')
        sns.scatterplot(x = df.x, y = df.y, s = 100, alpha = 0.2, color = 'w')
        sns.scatterplot(df_with_time_claster.mean_x_k, df_with_time_claster.mean_y_k, hue = df_with_time_claster.klaster, 
                        palette = 'dark', style = df_with_time_claster.time_klaster, legend = False, alpha = 0.15)

        for line in range(0, df_to_R.shape[0]):
            plt.text(x = df_to_R.x[line]-40, y = df_to_R.y[line]+5, s = df_to_R.index[line], 
                     fontdict = dict(color = 'red' if line % 2 == 0 else 'blue', size = 15))
            #plt.annotate(text='', xy=(df_to_R.x[line],df_to_R.y[line]), xytext=(df_to_R.x[line]-0.01,df_to_R.y[line]-0.01), arrowprops=dict(arrowstyle='->'), size = 100)
        ax.imshow(img)

        display_axes = fig.add_axes([0.78,0.13,0.1,0.1], projection='polar')
        display_axes._direction = 2*np.pi
        norm = mpl.colors.Normalize(0.0, 2*np.pi)
        cb = mpl.colorbar.ColorbarBase(display_axes, cmap=cmap,
                                           norm=norm,
                                           orientation='horizontal')
        cb.outline.set_visible(False)                                 
        display_axes.set_axis_off()

        plt.show()
        
        name = obiekt + '_bundle_transparent_' + group + '.jpg'
        fig.savefig(name, transparent=True)



        fig, ax = plt.subplots(figsize=(15, 10), dpi=100)

        for i in range(len(nazwa_1.edges)):
            ax.plot(nazwa_1.edges['x'][i], nazwa_1.edges['y'][i], color = nazwa_1.edges['color'][i], alpha = 0.7, linewidth = 4)
            ax.plot(nazwa_1.edges['x'][i], nazwa_1.edges['y'][i], color = 'w', alpha = 0.3, linewidth = 2)
            arrows = nazwa_1.edges['arrows'][i]
            for j in range(2):
                ax.plot(arrows[j][0], arrows[j][1], color = nazwa_1.edges['color'][i], linewidth = 4)
                ax.plot(arrows[j][0], arrows[j][1], color = 'w', alpha = 0.3, linewidth = 2)

        sns.scatterplot(x = df.x, y = df.y, s = 2000, alpha = 0.9, color = 'g', ax = ax)
        sns.scatterplot(x = df.x, y = df.y, s = 100, alpha = 0.7, color = 'w', ax = ax)
        sns.scatterplot(df_with_time_claster.mean_x_k, df_with_time_claster.mean_y_k, hue = df_with_time_claster.klaster, 
                        palette = 'dark', style = df_with_time_claster.time_klaster, legend = False, ax= ax, alpha = 0.1)

        for line in range(0, df_to_R.shape[0]):
            ax.text(x = df_to_R.x[line]-40, y = df_to_R.y[line]+5, s = df_to_R.index[line], 
                     fontdict = dict(color = 'red' if line % 2 == 0 else 'blue', size = 15))


        display_axes = fig.add_axes([0.78,0.13,0.1,0.1], projection='polar')
        display_axes._direction = 2*np.pi
        norm = mpl.colors.Normalize(0.0, 2*np.pi)
        cb = mpl.colorbar.ColorbarBase(display_axes, cmap=cmap,
                                           norm=norm,
                                           orientation='horizontal')
        cb.outline.set_visible(False)                                 
        display_axes.set_axis_off()


        ax.imshow(img, extent=[200, 1700, 0, 1050])
        plt.show()

        name = obiekt + '_bundle_' + group + '.jpg'
        fig.savefig(name)